<a href="https://colab.research.google.com/github/DLPY/Classification_Session_1/blob/main/ChurnModelligEDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



Customer Churn Prediction based on Banking Account Data
Detail on Data: https://www.kaggle.com/shrutimechlearn/churn-modelling

TODO: Download source data from Github
!wget https://raw.githubusercontent.com/DLPY/Regression-Session-2/master/Data/SydneyHousePrices.csv

Table of Contents:
Section 1 - Import the Libs and load the dataframe
Section 2 - EDA
Section 3 - Appendix

# # Import necessary packages for performing EDA

In [ ]:
import seaborn as sns #Visualization Lib
import matplotlib.pyplot as plt #Visualization Lib
import pandas as pd #Data manipulation lib
import numpy as np #mathamatical functions

In [ ]:
# Read data from csv file into Pandas dataframe
data = pd.read_csv(r'C:\Users\srajsrivastava\Downloads\Churn Modelling\Churn_Modelling.csv')

# Exploratory Data Analysis (EDA)
Perform data analysis, cleaning and transformation.

In [ ]:
# Display the count of rows and columns.
data.shape
#(10000 rows, 14 columns)

This data set contains details of a bank's customers and the target variable is a binary variable reflecting the fact whether the customer left the bank (closed their account) or they continue to be a customer.

Here we have 13 feature columns and Exited is a target column.

Row Numbers: Row Numbers from 1 to 10000.

CustomerId: Unique Ids for bank customer identification.

Surname: Customer's last name.

CreditScore: Credit score of the customer.

Geography: The country from which the customer belongs(Germany/France/Spain).

Gender: Male or Female.

Age: Age of the customer.

Tenure: Number of years for which the customer has been with the bank. Balance: Bank balance of the customer.

NumOfProducts: Number of bank products the customer is utilising.

HasCrCard: Binary Flag for whether the customer holds a credit card with the bank or not(0=No, 1=Yes).

IsActiveMember: Binary Flag for whether the customer is an active member with the bank or not(0=No, 1=Yes).

EstimatedSalary: Estimated salary of the customer in Euro.

Exited: Binary flag 1 if the customer closed account with bank and 0 if the customer is retained(0=No, 1=Yes).

In [ ]:
# Review a small sample of the data.
data.head()

In [ ]:
#Checking for null values
data.isnull().sum()

In [ ]:
# Detailed overview of the dataframe itself.
data.info()

From the above:
    Notice that there are no Non-Null Count values, e.g. there are no missing values in this data set.
    Also notice the different types of data: integer (int64), float (float64), and text (object).
    Geography and Gender need to be converted from a text value to a numeric value for modelling.

In [ ]:
# remove duplicates, if any
data = data[~data.duplicated()] 
data.shape
#No duplicate values

## Transformation

Encoding the categorical variables - Change the text into numbers
Review the unique values in the Geography column.

In [ ]:
data.Geography.unique()

In [ ]:
data.Gender.unique()

Convert the categorical values into numeric categorical labels so that this data can be reviewed in the EDA, e.g. correlations.

In [ ]:
#Decision tree needs data to be numeric
data['CountryCode'] = data['Geography'].astype('category').cat.codes
data['GenderCode'] = data['Gender'].astype('category').cat.codes

Geography - (0:France, 1:Germany, 2:Spain)
Gender - (0:Female, 1:Male)

In [ ]:
data.head()

From the above, notice that:
    The Geography and Gender have been converted to numeric values.
    There are two new columns with these values: CountryCode and GenderCode.

Quick review - columns that are not useful:
CustomerId - a unique customer ID number.
RowNumber - This is simply a row number of the data.
Surname - does not add any strength as a model input.
Geography - this has been converted to a numeric value instead of text.
Gender - this has been converted to a numeric value instead of text.

## Choosing predictor variables and target variable for performing Classification
Target and Source variables

Target Variable: Exited
Predictor Variables: CreditScore, CountryCode, GenderCode, Age, Tenure, Balance, NumOfProducts, HasCrCard, IsActiveMember, EstimatedSalary

## Create a new dataframe that includes only the selected columns

In [ ]:
df = data[['CreditScore', 'CountryCode', 'GenderCode', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
             'IsActiveMember', 'EstimatedSalary', 'Exited']]

In [ ]:
df.head()

### Investigate correlation in the new dataframe.
Pandas has a built-in correlation function. The df_new dataframe will be used to examine correlation.

In [ ]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
# Variables correlated to Exited.
df.corr()['Exited'].sort_values().drop('Exited').plot(kind='barh');

From the above, notice that:
    There are 10 variables that have varying degrees of correlation with 'Exited'.
    These should be examined closer, to try and get a better understanding of the correlation.

# Visualization

In [ ]:
#Summary of Exited and Non-Exited
sns.countplot(x='Exited', data = df)

From the above, notice that:
There are many more customers that have not 'Exited'. This is referred to as 'class imbalance'.

## High impact variables (Age, IsActiveMember)

In [ ]:
# Split the group into two cohorts for further analysis
Non_Exited = df[df['Exited'] == 0]
Exited = df[df['Exited'] == 1]
plt.subplots(figsize=(10, 5))
sns.distplot(Non_Exited['Age'])
sns.distplot(Exited['Age'])
plt.title('Age: Exited vs Non Exited')
plt.legend([0, 1], title='Exited')
plt.ylabel('percentage');

In [ ]:

cross_tab_prop = pd.crosstab(index=data['IsActiveMember'],
                             columns=data['Exited'],
                             normalize="index")
# Examine customers by country.
f, ax=plt.subplots(1, 2, figsize=(10, 2))
data['IsActiveMember'].value_counts().plot.bar(color=['#CD7F32','#FFDF00','#D3D3D3'], ax=ax[0])
ax[0].set_title('Count of customers by Active Members')
ax[0].set_ylabel('count')
cross_tab_prop.plot(kind='bar', stacked=True, colormap='tab10', figsize=(10, 2), ax=ax[1])
ax[1].set_title('Active Members: Exited vs Non Exited')
ax[1].set_ylabel('count');

# Appendix

## Low Impact variables (HasCreditCard, Estimated Salary)

In [ ]:
cross_tab_prop = pd.crosstab(index=data['HasCrCard'],
                             columns=data['Exited'],
                             normalize="index")
# Examine customers by country.
f, ax=plt.subplots(1, 2, figsize=(10, 5))
data['HasCrCard'].value_counts().plot.bar(color=['#CD7F32','#FFDF00','#D3D3D3'], ax=ax[0])
ax[0].set_title('Count of customers by HasCrCard')
ax[0].set_ylabel('count')
cross_tab_prop.plot(kind='bar', stacked=True, colormap='tab10', figsize=(10, 5), ax=ax[1])
ax[1].set_title('HasCrCard: Exited vs Non Exited')
ax[1].set_ylabel('count');

In [ ]:
plt.subplots(figsize=(10, 5))
sns.distplot(Non_Exited['EstimatedSalary'])
sns.distplot(Exited['EstimatedSalary'])
plt.title('EstimatedSalary: Exited vs Non Exited')
plt.legend([0, 1], title='Exited')
plt.ylabel('percentage');plt.figure(figsize=(18,8))


In [ ]:
cross_tab_prop = pd.crosstab(index=data['Geography'],
                             columns=data['Exited'],
                             normalize="index")
# Examine customers by country.
f, ax=plt.subplots(1, 2, figsize=(10, 2))
data['Geography'].value_counts().plot.bar(color=['#CD7F32','#FFDF00','#D3D3D3'], ax=ax[0])
ax[0].set_title('Count of customers by countries')
ax[0].set_ylabel('count')
cross_tab_prop.plot(kind='bar', stacked=True, colormap='tab10', figsize=(10, 2), ax=ax[1])
ax[1].set_title('Countries: Exited vs Non Exited')
ax[1].set_ylabel('count');

From the above, notice that:
    France has the largest number of customers
    Germany and Spain have a similar count of customers.
    Germany has the highest proportion of customers that 'Exited'.

In [ ]:
# Compare exited by age and geography.
sns.swarmplot(y='Age', x='Geography', hue='Exited', data=data);

Observations
    Germany has the highest proportion of churn (Lowest percentage of Non-exited customers and Highest percentage of exited         customers)
    Customers across all countries tend to have a higher tendency to churn after 40 years of age.

In [ ]:
# Examine customers by gender.
f, ax=plt.subplots(1, 2, figsize=(10, 2))
data['Gender'].value_counts().plot.bar(color=['#CD7F32','#FFDF00','#D3D3D3'], ax=ax[0])
ax[0].set_title('Count of customer by gender')
ax[0].set_ylabel('count')
cross_tab_prop = pd.crosstab(index=data['Gender'],
                             columns=data['Exited'],
                             normalize="index")
cross_tab_prop.plot(kind='bar', 
                    stacked=True, 
                    colormap='tab10', 
                    figsize=(10, 2), ax=ax[1])
ax[1].set_title('Gender: Exited vs Non Exited')
ax[1].set_ylabel('count');

In [ ]:
sns.swarmplot(y='Age', x='Gender', hue='Exited', data=data);

In [ ]:
# Examine customers by Age.
f, ax=plt.subplots(1, 2, figsize=(10, 4))
sns.boxplot(data['Age'], ax=ax[0])
ax[0].set_title('Count of customer by gender')
ax[0].set_ylabel('count')
sns.scatterplot(data=df, x='Age', y='Balance', hue='Exited', ax=ax[1])
ax[1].set_title('Age: Exited vs Non Exited')
ax[1].set_ylabel('Balance');

In [ ]:
sns.boxplot(data['CreditScore'])

From above we can see that majority of the people have a credit score between 590 to 720
Outliers are data points that are far from other data points. In other words, they’re unusual values in a dataset

From above, notice that there are three countries represented within the data set.

In [ ]:
cross_tab_prop = pd.crosstab(index=data['Age'],
                             columns=data['Exited'],
                             normalize="index")
cross_tab_prop.plot(kind='bar', 
                    stacked=True, 
                    colormap='tab10', 
                    figsize=(19, 6))

plt.legend(loc="upper left", ncol=2)
plt.xlabel("Age")
plt.ylabel("Exited")
plt.show()

In [ ]:
sns.countplot(x='Tenure', data = data)

In [ ]:
cross_tab_prop = pd.crosstab(index=data['Tenure'],
                             columns=data['Exited'],
                             normalize="index")
cross_tab_prop.plot(kind='bar', 
                    stacked=True, 
                    colormap='tab10', 
                    figsize=(10, 6))

plt.legend(loc="upper left", ncol=2)
plt.xlabel("Tenure")
plt.ylabel("Exited")
plt.show()

In [ ]:
sns.boxplot(data['Balance'])

In [ ]:
Non_Exited = data[data['Exited']==0]
Exited = data[data['Exited']==1]
plt.figure(figsize=(18,8))
p1=sns.kdeplot(Non_Exited['Balance'], shade=True, color="r")
p1=sns.kdeplot(Exited['Balance'], shade=True, color="b");
plt.title('Account Balance: Exited vs Non-Exited')
plt.legend([0,1],title='Exited');

In [ ]:
sns.countplot(x = 'NumOfProducts', data = data)

In [ ]:
pd.crosstab(data.NumOfProducts,data.Exited,margins=True).style.background_gradient(cmap='OrRd')

In [ ]:
cross_tab_prop = pd.crosstab(index=data['NumOfProducts'],
                             columns=data['Exited'],
                             normalize="index")
cross_tab_prop.plot(kind='bar', 
                    stacked=True, 
                    colormap='tab10', 
                    figsize=(19, 6))

plt.legend(loc="upper left", ncol=2)
plt.xlabel("NumOfProducts")
plt.ylabel("Exited")
plt.show()

In [ ]:
sns.countplot(x = 'HasCrCard', data = data)
cross_tab_prop = pd.crosstab(index=data['HasCrCard'],
                             columns=data['Exited'],
                             normalize="index")
cross_tab_prop.plot(kind='bar', 
                    stacked=True, 
                    colormap='tab10', 
                    figsize=(5, 4))

plt.legend(loc="upper left", ncol=2)
plt.xlabel("HasCrCard")
plt.ylabel("Exited")
plt.show()

In [ ]:
cross_tab_prop = pd.crosstab(index=data['HasCrCard'],
                             columns=data['Exited'],
                             normalize="index")
cross_tab_prop.plot(kind='bar', 
                    stacked=True, 
                    colormap='tab10', 
                    figsize=(19, 6))

plt.legend(loc="upper left", ncol=2)
plt.xlabel("HasCrCard")
plt.ylabel("Exited")
plt.show()

In [ ]:
plt.figure(figsize=(18,8))
plt.hist(x='EstimatedSalary',bins=100,data=Non_Exited,edgecolor='black',color='red')
plt.hist(x='EstimatedSalary',bins=100,data=Exited,edgecolor='black',color='blue')
plt.title('Estimated salary: Exited vs Non-Exited')
plt.legend([0,1],title='Exited');